# KAIST AI605 Assignment 2: Text Retrieval
TA in charge: Yongrae Jo (yongrae@kaist.ac.kr)

**Due Date:** October 27 (Wed) 11:00pm, 2021

## Your Submission
If you are a KAIST student, you will submit your assignment via [KLMS](https://klms.kaist.ac.kr). If you are a NAVER student, you will submit via [Google Form](https://forms.gle/aGZZ86YpCdv2zEVt9). 

You need to submit both (1) a PDF of this notebook, and (2) a link to CoLab for execution (.ipynb file is also allowed).

Use in-line LaTeX (see below) for mathematical expressions. Collaboration among students is allowed but it is not a group assignment so make sure your answer and code are your own. Make sure to mention your collaborators in your assignment with their names and their student ids.

## Grading
The entire assignment is out of 20 points. You can obtain up to 5 bonus points (i.e. max score is 25 points). For every late day, your grade will be deducted by 2 points (KAIST students only). You can use one of your no-penalty late days (7 days in total). Make sure to mention this in your submission. You will receive a grade of zero if you submit after 7 days.


## Environment
You will only use Python 3.7 and PyTorch 1.9, which is already available on Colab:

In [ ]:
from platform import python_version
import torch
import numpy as np
from tqdm import tqdm
import time

print("python", python_version())
print("torch", torch.__version__)

python 3.7.12
torch 1.9.0+cu111


You will use two datasets, namely SQuAD, for retrieval. Note that this is a MRC dataset but we will view them as retrieval task by trying to find the correct document corresponding to the question among all the documents in the **validation** data. 

In [ ]:
!pip install -q datasets

     |████████████████████████████████| 290 kB 5.4 MB/s 
     |████████████████████████████████| 243 kB 43.5 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 125 kB 59.8 MB/s 
     |████████████████████████████████| 1.3 MB 45.7 MB/s 
     |████████████████████████████████| 271 kB 48.9 MB/s 
     |████████████████████████████████| 160 kB 44.1 MB/s 


In [ ]:
from datasets import load_dataset
from pprint import pprint

squad_dataset = load_dataset('squad')
pprint(squad_dataset['train'][0]) # 'context' contains the document

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did t

## 1. Measuring Similarity
We discussed in Lecture 08 that there are several ways to measure similarity between two vectors, such as L2 (Euclidean) distance, L1 (Manhattan) distance, inner product, and cosine distance. Here, except for inner product, all other measures are *metric* (see *Definition* at https://en.wikipedia.org/wiki/Metric_(mathematics)).

> **Problem 1.1** *(2 points)* Using the definition of metric above, prove that L1 distance is a metric.

**(Proof)**
For L1 distance, for $x,y \in \mathbb{R}^n$, $d(x,y) = \sum_{i=1}^n|x_i-y_i|$.

($i$) 
\begin{align}
d(x,y) &= \sum_{i=1}^n|x_i-y_i| = 0 \\
      &\iff |x_i-y_i| = 0 ~~\text{for every }i\in [n] \\
      &\iff x_i = y_i ~~\text{for every }i\in [n] \\
      &\iff x=y ~~ \text{(identity of indiscernibles)}
\end{align}

($ii$)  
\begin{align}
d(x,y) &= \sum_{i=1}^n|x_i-y_i| \\
&= \sum_{i=1}^n|y_i-x_i| \\
&= d(y,x) ~~ \text{(symmetry)}
\end{align}

($iii$)
\begin{align}
d(x,y) &= \sum_{i=1}^n|x_i-y_i| \\
&= \sum_{i=1}^n|x_i-z_i+z_i-y_i| \\
&\leq \sum_{i=1}^n|x_i-z_i| + \sum_{i=1}^n|z_i-y_i| \\
&= d(x,z) + d(z,y) ~~ \text{(triangular inequality)}
\end{align}

Therefore, L1 distance is a metric.

> **Problem 1.2** *(2 points)* Prove that negative inner product is NOT a metric.

**(Proof)**
Negative inner product is not a metric, since it does not satisfy the identity of indiscernibles. If $x=[1,0]$ and $y=[0,1]$, negative inner product $d(x,y)=0$, but $x\neq y$.

> **Problem 1.3** *(2 points)* Prove that cosine distance (1 - cosine similarity) is a metric.

**(Proof)**
Cosine distance is **NOT** a metric. It does not even hold the identity of indiscernibles. If two vectors $x,y$ have same direction and different scale, e.g., $x=[1,0]$ and $y=[2,0]$, cosine disntance $d(x,y) = 1-\frac{x\cdot y}{\|x\|\cdot\|y\|}=0$, but $x\neq y$.

> **Problem 1.4 (bonus)** *(3 points)* Given a model that can perform nearest neighbor search in L2 space, can you modify your query and your key vectors to perform maximum inner product search? (Hint: Recall the difference between MIPS and L2 NNS in Lecture 09. Can you modify key vectors so that the difference becomes 0?)

**(Proof)**
We can make L2 NNS equivalent to MIPS via **normalizing the query and key vectors.** In other words, 
\begin{align}
\arg \min_i \|q-k_i\|_2 
&= \arg \min_i \|q-k_i\|^2_2 \\
&= \arg \min_i \|q\|^2_2 + \|k_i\|^2_2 - 2\cdot q\cdot k_i \\ 
&= \arg \max_i q\cdot k_i 
\end{align}
where the last equality holds because $q$ and $k_i$ are normalized and the norm is constant as 1. 

We first create an abstract class for performing similarity search as follows (`raise NotImplementedError()` means you have to override these methods when you subclass the class):

In [ ]:
class SimilaritySearch(object):
  def __init__(self):
    raise NotImplementedError()

  def train(self, documents: list):
    raise NotImplementedError()

  #Add documents (a list of text)
  def add(self, documents: list):
    raise NotImplementedError()

  #Returns the indices of top-k documents among the added documents
  #that are most similar to the input query 
  def search(self, query: list, k: int) -> list:
    raise NotImplementedError()


You will use the same space-based tokenizer that you used in Assignment 1, with lowercasing to make it case insensitive.

## 2. Sparse Search



> **Problem 2.1** *(2 points)* We will first start with Bag of Words that we discussed in Lecture 08. Using the definition in the class (don't worry about the exact definition though), implement `BagOfWords` class that subclasses `SimilaritySearch` class.

In [ ]:
class BagOfWords(SimilaritySearch):
  def __init__(self, dataset):
    self.words = dict(UNK=0)
    self.documents = []
    self.dataset = dataset

    for i in range(len(dataset)):
      sentence_ = dataset[i]['context'].split(' ')
      self.train(sentence_)

    for i in range(len(dataset)):
      self.add(dataset[i]['context'].split(' '))

  def train(self, documents):
    for word_ in documents:
      word = word_.lower()
      if word not in self.words:
        self.words[word] = len(self.words)

  def add(self, documents):
    sentence_to_idx = {}
    for word_ in documents:
      word = word_.lower()
      if self.words[word] in sentence_to_idx:
        sentence_to_idx[self.words[word]] += 1
      else:
        sentence_to_idx[self.words[word]] = 1
    self.documents.append(sentence_to_idx)

  def search(self, query, k):
    query_bag_of_words = {0:0}
    for word_ in query:
      word = word_.lower()
      if word not in self.words:
        query_bag_of_words[0] += 1   # UNK
      elif word not in query_bag_of_words:
        query_bag_of_words[self.words[word]] = 1
      else:
        query_bag_of_words[self.words[word]] += 1 

    scores = []
    for doc in range(len(self.documents)):
      score = 0
      for text_idx in query_bag_of_words:
        if text_idx == 0:
          continue
        if text_idx in self.documents[doc]:
          score += query_bag_of_words[text_idx] * self.documents[doc][text_idx]
      scores.append(score)
    indices = np.argsort(scores)[-k:][::-1]

    return indices, scores

> **Problem 2.2** *(2 points)* Using the definition in Lecture 08 (don't worry about the exact definition though), implement `TFIDF` class that subclasses `BagOfWords` class.

In [ ]:
class TFIDF(BagOfWords):
  def __init__(self, dataset):
    self.words = dict(UNK=0)
    self.word_count = {0:0}
    self.word_idf = {0:0}
    self.documents = []
    self.dataset = dataset

    for i in range(len(dataset)):
      sentence_ = dataset[i]['context'].split(' ')
      self.train(sentence_)
    
    for idx in self.word_count:
      if self.word_count[idx] == 0:
        self.word_idf[idx] = 0
      else:
        self.word_idf[idx] = np.log(len(self.dataset) / self.word_count[idx])

    for i in range(len(dataset)):
      self.add(dataset[i]['context'].split(' '))

  def train(self, documents):
    ll = []
    for word_ in documents:
      word = word_.lower()
      if word not in self.words:
        self.words[word] = len(self.words)
        self.word_count[self.words[word]] = 0
      if word not in ll:
        self.word_count[self.words[word]] += 1
        ll.append(word)

  def add(self, documents):
    sentence_to_idx = {}
    num_words = len(documents)
    for word_ in documents:
      word = word_.lower()
      if self.words[word] in sentence_to_idx:
        sentence_to_idx[self.words[word]] += 1 / num_words * self.word_idf[self.words[word]]
      else:
        sentence_to_idx[self.words[word]] = 1 / num_words * self.word_idf[self.words[word]]
    self.documents.append(sentence_to_idx)

  def search(self, query, k):
    query_bag_of_words = {0:0}
    num_words = len(query)
    for word_ in query:
      word = word_.lower()
      if word not in self.words:
        continue   # UNK
      elif word not in query_bag_of_words:
        query_bag_of_words[self.words[word]] = 1 / num_words * self.word_idf[self.words[word]]
      else:
        query_bag_of_words[self.words[word]] += 1 / num_words * self.word_idf[self.words[word]]

    scores = []
    for doc in range(len(self.documents)):
      score = 0
      for text_idx in query_bag_of_words:
        if text_idx == 0:
          continue
        if text_idx in self.documents[doc]:
          score += query_bag_of_words[text_idx] * self.documents[doc][text_idx]
      scores.append(score)
    indices = np.argsort(scores)[-k:][::-1]

    return indices, scores

> **Problem 2.3** *(2 points)* Use `TFIDF` to masure the recall rate of the correct document when 10 documents (contexts) are retrieved (this is called **Recall@10**) in SQuAD **validation** set.

Recall@10 of TFIDF is 55.3%. (see the code)

In [ ]:
dataset = squad_dataset['validation']
tfidf = TFIDF(dataset)
recall = 0
for doc in tqdm(range(len(dataset))):
  question = dataset[doc]['question'].split(' ')
  indices,_ = tfidf.search(question, 10)
  if doc in indices:
    recall += 1
recall /= len(dataset)

print(recall)

100%|██████████| 10570/10570 [06:47<00:00, 25.94it/s]

0.5528855250709556


> **Problem 2.4 (bonus)** *(2 points)* Implement `BM25` that sublcasses `BagOfWords` and repeat Problem 2.3 for BM25.

Recall@10 of BM25 is 46.3%. (see the code)

In [ ]:
class BM25(BagOfWords):
  def __init__(self, dataset):
    self.words = dict(UNK=0)
    self.word_count = {0:0}
    self.word_idf = {0:0}
    self.documents = []
    self.document_length = []
    self.dataset = dataset

    for i in range(len(dataset)):
      sentence_ = dataset[i]['context'].split(' ')
      self.train(sentence_)
    
    for idx in self.word_count:
      if self.word_count[idx] == 0:
        self.word_idf[idx] = 0
      else:
        self.word_idf[idx] = np.log(len(self.dataset) / self.word_count[idx])

    for i in range(len(dataset)):
      self.add(dataset[i]['context'].split(' '))

  def train(self, documents):
    ll = []
    for word_ in documents:
      word = word_.lower()
      if word not in self.words:
        self.words[word] = len(self.words)
        self.word_count[self.words[word]] = 0
      if word not in ll:
        self.word_count[self.words[word]] += 1
        ll.append(word)

  def add(self, documents):
    sentence_to_idx = {}
    num_words = len(documents)
    for word_ in documents:
      word = word_.lower()
      if self.words[word] in sentence_to_idx:
        sentence_to_idx[self.words[word]] += 1 / num_words * self.word_idf[self.words[word]]
      else:
        sentence_to_idx[self.words[word]] = 1 / num_words * self.word_idf[self.words[word]]
    self.documents.append(sentence_to_idx)
    self.document_length.append(num_words)

  def search(self, query, k):
    query_bag_of_words = {0:0}
    num_words = len(query)
    for word_ in query:
      word = word_.lower()
      if word not in self.words:
        continue   # UNK
      elif word not in query_bag_of_words:
        query_bag_of_words[self.words[word]] = self.word_idf[self.words[word]]
      else:
        query_bag_of_words[self.words[word]] += self.word_idf[self.words[word]]

    scores = []
    k1, b = 1.5, 0.75
    avgdl = np.mean(self.document_length)
    for doc in range(len(self.documents)):
      score = 0
      for text_idx in query_bag_of_words:
        if text_idx == 0:
          continue
        if text_idx in self.documents[doc]:
          score += query_bag_of_words[text_idx] * (self.documents[doc][text_idx] * (k1+1)) \
                   / (self.documents[doc][text_idx] + k1 * (1-b + b * self.document_length[doc] / avgdl))
      scores.append(score)
    indices = np.argsort(scores)[-k:][::-1]

    return indices, scores

In [ ]:
dataset = squad_dataset['validation']
bm25 = BM25(dataset)
recall = 0
for doc in tqdm(range(len(dataset))):
  question = dataset[doc]['question'].split(' ')
  indices,_ = bm25.search(question, 10)
  if doc in indices:
    recall += 1
recall /= len(dataset)

print(recall)

100%|██████████| 10570/10570 [11:19<00:00, 15.55it/s]

0.46253547776726583


## 3. Dense Search

To obtain the embedding of each document and query, you will use GloVe embedding (Pennington et al., 2014). Go to https://nlp.stanford.edu/projects/glove/ and download `glove.6B.zip`. You are also welcome to use other more convenient ways to access the GloVe embeddings. You will compute the document's embedding by simply averaging the embeddings of all words in the document (same for the query), and then normalizing it. This way, inner product effectively becomes cosine similarity.

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-10-25 15:10:35--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-10-25 15:10:35--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.81MB/s    in 2m 52s  

2021-10-25 15:13:28 (4.77 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: 

> **Problem 3.1** *(3 points)* Implement `GloVeAverage` class that subclasses `SimilaritySearch` class, using PyTorch's tensor native operation for the dense search. 

In [ ]:
vocab = []
embedding = np.zeros((400000, 300), dtype=np.float32)
#embedding[0,:] = np.random.randn(300)  # random number for UNK
with open('./glove.6B.300d.txt') as f:
  for i, line in enumerate(f.readlines()):
    vocab.append(line.split(' ')[0])
    embedding[i] = np.array(line.split(' ')[1:], dtype=np.float32)
embedding = torch.from_numpy(embedding)
word2id = {word: id_ for id_, word in enumerate(vocab)}

In [ ]:
class GloVeAverage(SimilaritySearch):
  def __init__(self, dataset, voacb, embedding, word2id):
    self.dataset = dataset
    self.vocab = vocab
    self.embedding = embedding
    self.word2id = word2id
    self.documents = []

    for i in tqdm(range(len(dataset))):
      self.add(dataset[i]['context'].split(' '))

  def train(self, documents):
    pass

  def add(self, documents):
    sentence_to_idx = []
    ll = []
    for word_ in documents:
      word = word_.lower()
      if word in ll:
        continue
      if word not in self.vocab:
        continue
      idx = self.word2id[word]
      sentence_to_idx.append(idx)
      ll.append(word)
    emb = self.embedding[sentence_to_idx].mean(0)
    self.documents.append(emb)

  def search(self, query_set, k):
    start_time = time.time()
    query_emb_documents = []
    for doc in tqdm(range(len(query_set))):
      query = query_set[doc]['question'].split(' ')
      sentence_to_idx = []
      for word_ in query:
        word = word_.lower()
        if word not in self.vocab:
          continue
        idx = self.word2id[word]
        sentence_to_idx.append(idx)
      query_emb = self.embedding[sentence_to_idx].mean(0)
      query_emb_documents.append(query_emb)
    query_emb_documents = torch.nn.functional.normalize(torch.stack(query_emb_documents, 0), dim=1)
    
    documents = torch.nn.functional.normalize(torch.stack(self.documents, 0), dim=1)
    scores = torch.mm(query_emb_documents, documents.transpose(0, 1))
    indices = torch.topk(scores, k, dim=1)[1]
    
    end_time = time.time() - start_time
    print('Search Time: {}s'.format(end_time))
    return indices

> **Problem 3.2** *(2 points)* Use `GloVeAverage` to measure the recall at 10 for SQuAD validation dataset. How does it compare to TFIDF?

Recall@10 of GloveAverage is 31.5%. (see the code) TFIDF was better than GloVeAverage for SQuAD valiation set.

In [ ]:
dataset = squad_dataset['validation']
gloveaverage = GloVeAverage(dataset, vocab, embedding, word2id)
recall = 0
indices = gloveaverage.search(dataset, 10)
for i in range(len(indices)):
  if i in indices[i]:
    recall += 1
recall /= len(dataset)

print(recall)

100%|██████████| 10570/10570 [01:59<00:00, 88.78it/s]


Search Time: 121.20724582672119s
0.31494796594134344


> **Problem 3.3** *(2 points)* Implement `GloVeAverageFaiss` that subclasses `SimilaritySearch` and uses Faiss `IndexFlatIP` instead of PyTorch native tensor operation for search. Refer to the Faiss wiki (https://github.com/facebookresearch/faiss/wiki/Getting-started) for instructions.

In [ ]:
!pip install faiss-cpu
import faiss

     |████████████████████████████████| 8.4 MB 4.4 MB/s 


In [ ]:
class GloVeAverageFaiss(SimilaritySearch):
  def __init__(self, dataset, voacb, embedding, word2id):
    self.dataset = dataset
    self.vocab = vocab
    self.embedding = embedding.numpy()
    self.d = self.embedding.shape[1]
    self.index = faiss.IndexFlatIP(self.d)
    self.word2id = word2id
    self.documents = []

    for i in tqdm(range(len(dataset))):
      self.add(dataset[i]['context'].split(' '))
    # print(self.index.ntotal)

  def train(self, documents):
    pass

  def add(self, documents):
    sentence_to_idx = []
    ll = []
    for word_ in documents:
      word = word_.lower()
      if word in ll:
        continue
      if word not in self.vocab:
        continue
      idx = self.word2id[word]
      sentence_to_idx.append(idx)
      ll.append(word)
    emb = self.embedding[sentence_to_idx].mean(0, keepdims=True)
    emb = emb / np.linalg.norm(emb, 2, 1, keepdims=True)
    self.index.add(emb)

  def search(self, query_set, k):
    start_time = time.time()
    query_emb_documents = []
    for doc in tqdm(range(len(query_set))):
      query = query_set[doc]['question'].split(' ')
      sentence_to_idx = []
      for word_ in query:
        word = word_.lower()
        if word not in self.vocab:
          continue
        idx = self.word2id[word]
        sentence_to_idx.append(idx)
      query_emb = self.embedding[sentence_to_idx].mean(0)
      query_emb_documents.append(query_emb)

    query_emb_documents = np.stack(query_emb_documents, 0)
    query_emb_documents = query_emb_documents / np.linalg.norm(query_emb_documents, 2, 1, keepdims=True)
    
    D, I = self.index.search(query_emb_documents, k)
    end_time = time.time() - start_time
    print('Search Time: {}s'.format(end_time))
    return I

> **Problem 3.4** *(1 points)* Compare the speed between `GloVeAverage` and `GloVeAverageFaiss` on SQuAD. To make the measurement accurate, perform search many times (at least more than 1000) and take the average.

GloVeAverage vs. GloVeAverageFaiss : 121.2s vs. 119.9s (over 10000 searches)

The procedure of adding query vectors to arrays was a bottleneck, so there was not much difference in total time.

In [ ]:
dataset = squad_dataset['validation']
gloveaveragefaiss = GloVeAverageFaiss(dataset, vocab, embedding, word2id)
recall = 0
indices = gloveaveragefaiss.search(dataset, 10)
for i in range(len(indices)):
  if i in indices[i]:
    recall += 1
recall /= len(dataset)

print(recall)

100%|██████████| 10570/10570 [01:58<00:00, 89.13it/s]


Search Time: 119.89045238494873s
0.31494796594134344
